# classroom data

In [1]:
import pickle
from data.class_data import loaddata
import numpy as np
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch
from math import ceil, floor
import os
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
class ClassroomDataset(Dataset):
    
    @staticmethod
    def standardize_image(image):
        expected_shape = (50,50)
        padding = [(0,0), (0,0)]
        cropping = [(0,50), (0, 50)]
        for d in [0,1]:
            exp = expected_shape[d]
            act = image.shape[d]
            diff = exp - act
            if diff > 0:
                padding[d] = (ceil(diff / 2), floor(diff / 2))
            elif diff < 0:
                diff *= -1
                cropping[d] = (ceil(diff / 2), 50 + floor(diff / 2))
        image = np.pad(image, padding, 'constant', constant_values=0)
        y = cropping[0]
        x = cropping[1]
        image = image[y[0]:y[1], x[0]:x[1]]
        return image
    
    def to_include(self, i):
        ret = False
        mx = self.range_10[1]
        mn = self.range_10[0]
        if i % 10 < mx and i % 10 >= mn:
            ret = True
        return ret
    
    def __init__(self, range_10=[0, 10]):
        path = "data/class_data/train_data.pkl"
        cls_data = loaddata.load_pkl(path)
        labels = np.load("data/class_data/finalLabelsTrain.npy") 
        self.data = []
        self.range_10 = range_10
        gpu = torch.cuda.is_available()
#         gpu = False
        for i in range(len(cls_data)):
            if self.to_include(i): # train set
                image = np.array(cls_data[i])
                image = image.astype(int)
                image = self.standardize_image(image)
                image = torch.from_numpy(np.array([image])).float()
                label = torch.tensor(int(labels[i]))
                if gpu:
                    image = image.cuda()
                    label = label.cuda()
                pair = (image, label)
                self.data.append(pair)
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)

In [3]:
cls_data = ClassroomDataset(range_10=[5,10])
i = 0
cls_data.__len__()

3200

# our data 2

# our data 1

In [4]:
transform = transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()])
Parth_preprocessed_dataset = ImageFolder(
    root="data/Parth_processed/",
    transform=transform
)

# EMNIST